In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import datetime

%load_ext tensorboard

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
SHUFFLE_BUFFER_SIZE = 1000
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [4]:
def preprocessing(img,label):
    img = tf.cast(img, tf.float32)
    img = (img/128) -1
    label = tf.one_hot(label, depth=10)
    return img, label


def load_and_prep_cifar(batch_size, shuffle_buffer_size):
    (train_data, test_data), ds_info = tfds.load(name='cifar10',
                                                split=['train','test'],
                                                as_supervised=True,
                                                with_info=True)
    train_data = train_data.map(lambda img, label: preprocessing(img,label))
    test_data = test_data.map(lambda img, label: preprocessing(img,label))
    train_data = train_data.shuffle(shuffle_buffer_size).batch(batch_size).prefetch(20)
    test_data = test_data.batch(batch_size).prefetch(20)
    return train_data, test_data, ds_info

In [5]:
class CNN_Model(tf.keras.Model):
    def __init__(self, blocks: int, size_of_blocks: [int]):
        if blocks != len(size_of_blocks):
            raise ValueError("The number of blocks and the size of blocks must be the same")
        
        super(CNN_Model, self).__init__()
        
        self.optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=LEARNING_RATE)
        
        self.metrics_list = [
                        tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
                       ]
        
        self.loss_f = tf.keras.losses.CategoricalCrossentropy()
        
        self.model_layers = []
        
        for block in range(blocks):
            self.model_layers.append(tf.keras.layers.Conv2D(activation="relu", filters=size_of_blocks[block], kernel_size=3, padding="same"))
            self.model_layers.append(tf.keras.layers.Conv2D(activation="relu", filters=size_of_blocks[block], kernel_size=3, padding="same"))
            self.model_layers.append(tf.keras.layers.Conv2D(activation="relu", filters=size_of_blocks[block], kernel_size=3, padding="same"))
            self.model_layers.append(tf.keras.layers.AvgPool2D(pool_size=(2,2), strides=(2,2)))
        self.global_pooling = tf.keras.layers.GlobalAveragePooling2D()
        self.output_layer = tf.keras.layers.Dense(10, activation='softmax')
       
    @tf.function
    def call(self, x):
        for layer in self.model_layers:
            x = layer(x)
        x = self.global_pooling(x)
        output = self.output_layer(x)
        return output
    
    def reset_metrics(self):
        for metric in self.metrics_list:
            metric.reset_states()
            
            
    @tf.function
    def train_step(self, data):
        x, target = data
        with tf.GradientTape() as tape:
            pred = self(x)
            loss = self.loss_f(target, pred) + tf.reduce_sum(self.losses)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(grads_and_vars=zip(gradients, self.trainable_variables))
        
        self.metrics[0].update_state(loss)
        
        for metric in self.metrics[1:]:
            metric.update_state(target, pred)
        return {m.name: m.result() for m in self.metrics}
    
    @tf.function
    def test_step(self, data):
        x, target = data
        pred = self(x)
        loss = self.loss_f(target, pred) + tf.reduce_sum(self.losses)
        
        self.metrics[0].update_state(loss)
        
        for metric in self.metrics[1:]:
            metric.update_state(target, pred)
        return {m.name: m.result() for m in self.metrics}

In [6]:
model = CNN_Model(4, [5,10,32,64])

In [7]:
# Define where to save the log
config_name= "Run-1"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_path = f"logs/{config_name}/{current_time}/train"
test_log_path = f"logs/{config_name}/{current_time}/val"

# log writer for training metrics
train_summary_writer = tf.summary.create_file_writer(train_log_path)

# log writer for validation metrics
test_summary_writer = tf.summary.create_file_writer(test_log_path)

In [8]:
import tqdm

def training_loop(model, train, test, epochs, train_summary_writer, test_summary_writer):
    for epoch in range(epochs):
        
        #Training
        for data in tqdm.tqdm(train, position=0, leave=True):
            metrics = model.train_step(data)
            with train_summary_writer.as_default():
                for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
        
        # print the metrics
        print([f"{key}: {value.numpy()}" for (key, value) in metrics.items()])
        
        # reset all metrics (requires a reset_metrics method in the model)
        model.reset_metrics()

        # Validation:
        for data in test:
            metrics = model.test_step(data)

            # logging the validation metrics to the log file which is used by tensorboard
            with test_summary_writer.as_default():
                for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        print([f"val_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # reset all metrics
        model.reset_metrics()
        print("\n")

In [10]:
train, test, info = load_and_prep_cifar(batch_size=BATCH_SIZE, shuffle_buffer_size=SHUFFLE_BUFFER_SIZE)

training_loop(model, train, test, 5, train_summary_writer, test_summary_writer)

100%|██████████| 782/782 [00:20<00:00, 37.45it/s]


['loss: 1.3209162950515747', 'acc: 0.5230748057365417']
['val_loss: 1.3063057661056519', 'val_acc: 0.5285999774932861']


100%|██████████| 782/782 [00:19<00:00, 40.66it/s]


['loss: 1.2064121961593628', 'acc: 0.5674600005149841']
['val_loss: 1.1777360439300537', 'val_acc: 0.5741000175476074']


100%|██████████| 782/782 [00:19<00:00, 39.86it/s]


['loss: 1.1282669305801392', 'acc: 0.5958399772644043']
['val_loss: 1.1595337390899658', 'val_acc: 0.5866000056266785']


100%|██████████| 782/782 [00:19<00:00, 39.90it/s]


['loss: 1.0572736263275146', 'acc: 0.6214600205421448']
['val_loss: 1.1001906394958496', 'val_acc: 0.6093999743461609']


100%|██████████| 782/782 [00:19<00:00, 40.56it/s]


['loss: 1.0036523342132568', 'acc: 0.6421200037002563']
['val_loss: 1.0338387489318848', 'val_acc: 0.632099986076355']


In [11]:
%tensorboard --logdir logs

Launching TensorBoard...

In [ ]:
'''

def train(num_epochs, batchsize, shuffle_buffer_size,lr):
    model = CNN_Model(4, [5,10,32,64])
    train, test, info = load_and_prep_cifar(batch_size=batchsize, shuffle_buffer_size=shuffle_buffer_size)
    initial_learning_rate = 0.001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=50000, decay_rate=0.96, staircase=True)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule)
    loss_f = tf.keras.losses.CategoricalCrossentropy()
    for epoch in range(num_epochs):
        losses = []
        for x, t in train:
            with tf.GradientTape() as tape:
                pred = model(x)
                l = loss_f(t, pred)
            gradients = tape.gradient(l, model.trainable_variables)
            optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))
            losses.append(l.numpy())
        print(np.mean(losses))
'''


    

In [ ]:
# train(15, BATCH_SIZE, SHUFFLE_BUFFER_SIZE,lr=0.001)